# 허깅페이스로 나만의 요약 모델 만들기 튜토리얼 자료

**진행자:** [윤주성](https://www.linkedin.com/in/joosung-yoon/)<br>
**참고자료:** [Abstractive Summarization with Hugging Face Transformers](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/t5_hf_summarization.ipynb)

## Introduction
자동 요약은 NLP에서 주요 문제중 하나 입니다. 요약을 위한 챌린지는 중요한 컨텐츠가 무엇인지 파악해야하고, 새로운 표현으로 문장을 생성해야합니다.

이번 실습에서는 케라스 NLP와 허깅페이스를 활용하여 single-document에 대한 *Abstractive Summarization* task를 다뤄보고자 합니다.

사용할 모델은 [Transformer]((https://arxiv.org/abs/1706.03762)) 기반 모델인 [Text-to-Text Transfer Transformer (`T5`)](https://arxiv.org/abs/1910.10683)를 사용할 예정입니다.
T5는 encoder-decoder 구조를 갖는 모델로 요약, 번역등 다양한 task에서 활용됩니다.

이번 실습에서는 [문서요약 텍스트 샘플(샘플)](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97) 이라는 간단한 한글 텍스트 코퍼스를 가지고 텍스트 생성 모델을 학습 합니다. 

- 텍스트 전처리를 위한 토크나이저 생성 및 동작 방식 이해 
- T5 텍스트 생성 모델 구성과 학습에 대한 이해
- 학습한 T5 모델에 대한 텍스트 생성 동작 방식에 대한 이해

**시작하기 전에 주의할 점**: Colab에서 실습하시게 되면 세션 설정에 반드시 GPU runtime을 설정해주시기 바랍니다.

## Setup
### Installing the requirements

In [ ]:
!pip install transformers==4.20.0
!pip install nltk
!pip install rouge-score
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 33.6 MB/s 
     |████████████████████████████████| 163 kB 61.7 MB/s 
     |████████████████████████████████| 6.6 MB 57.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=6e48f955ca4fa756f1269d0b821682ffa0fc21cffbaeebb24b078351d7e5669a
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 33.3 MB/s 
     |████████████████████████████████| 5.9 MB 49.1 MB/s 
     |██████████

### Importing the necessary libraries

In [ ]:
import os
import logging
from pprint import pprint
import json
import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 에러 메세지만 로깅
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Define certain variables

In [ ]:
# train과 test 데이터셋으로 나누는 비율
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 512  # encoder에 들어갈 max input 길이
MIN_TARGET_LENGTH = 5  # decoder에 들어갈 min input 길이
MAX_TARGET_LENGTH = 128  # decoder에 들어갈 max input 길이
BATCH_SIZE = 8  # 모델 학습에 사용할 batch siz 크기
LEARNING_RATE = 2e-5  # 모델 학습에 사용할 learning rate
MAX_EPOCHS = 1  # 모델 학습에 사용할 epoch수

# Hugging Face Model Hub로 부터 가져올 모델명
MODEL_CHECKPOINT = "psyche/KoT5-summarization"

## Load the dataset
[문서요약 텍스트 샘플(샘플)](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97) 이라는 간단한 한글 텍스트 코퍼스를 다운로드 하곘습니다.   
전체 데이터셋은 AI Hub에서 다운받으실 수 있습니다.
다운 받은 데이터셋을 [Hugging Face Datasets](https://github.com/huggingface/datasets) 포멧으로 변환합니다.  
이미 Hugging Face hub에 올라온 데이터셋의 경우 `load_dataset` 함수를 통해 바로 사용할 수 있습니다.

In [ ]:
!gdown 1S5kUCc-u-F2w5JOgS81w2ht6Wmcjo7-y

Downloading...
From: https://drive.google.com/uc?id=1S5kUCc-u-F2w5JOgS81w2ht6Wmcjo7-y
To: /content/sample.jsonl
100% 8.64M/8.64M [00:00<00:00, 150MB/s]


In [ ]:
!ls

sample_data  sample.jsonl


In [ ]:
!apt-get install jq
!head -n 1 ./sample.jsonl | jq '.'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libjq1 libonig4
The following NEW packages will be installed:
  jq libjq1 libonig4
0 upgraded, 3 newly installed, 0 to remove and 27 not upgraded.
Need to get 276 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libonig4 amd64 6.7.0-1 [119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libjq1 amd64 1.5+dfsg-2 [111 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 jq amd64 1.5+dfsg-2 [45.6 kB]
Fetched 276 kB in 1s (206 kB/s)
Selecting previously unselected package libonig4:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../libo

데이터의 키 값을 article_original은 document로, abstractive는 summary로 변경합니다.

In [ ]:
from datasets import Dataset, load_dataset
ds_path = "./sample.jsonl"
_id, document, summary = [], [], []

with open(ds_path, 'r') as f:
  while True:
    line = f.readline()
    if not line: break
    doc = json.loads(line)
    _id.append(doc['id'])
    document.append(" ".join(doc['article_original']))
    summary.append(doc['abstractive'])

# raw_datasets = load_dataset("xsum", split="train")
raw_datasets = Dataset.from_dict({"id":_id,
                                  "document":document,
                                  "summary":summary})
pprint(raw_datasets[0])

{'document': '보조금 집행 위법행위·지적사례 늘어 특별감사반, 2017~2018년 축제 점검 충주시가 민간에게 지원되는 보조사업의 '
             '대형축제와 관련해 선정·집행·정산 등 운영실태 전반에 대한 자체 감사를 실시할 계획이라고 밝혔다. 이는 최근 '
             '민간보조사업의 증가와 더불어 보조금 집행관리에 대한 위법 부당 행위와 지적사례가 지속적으로 증가함에 따라, 감사를 '
             '통해 취약요인을 점검해 올바른 보조금 사용 풍토를 정착시키겠다는 취지다. 시는 감사담당관실과 기획예산과 보조금 관련 '
             '주무관으로 특별감사반을 편성해 2017년부터 2018년까지 집행된 축제성 보조금 집행에 대한 철저한 점검과 감사를 '
             '통해 부정 수급 및 부정 집행이 확인되면 엄정한 조치를 취할 방침이다. 시는 지난 15일부터 25일까지 10일간의 '
             '사전감사를 통해 보조금 실태를 파악한 후, 8월15일까지 세부감사를 진행할 예정이라고 전했다. 축제성 관련 부정수급 '
             '유형을 보면 허위·기타 부정한 방법으로 보조금 신청, 사업 실적을 부풀려 보조금을 횡령·편취, 보조금 교부 목적과 '
             '다른 용도로 집행, 보조금으로 취득한 재산에 대해 지자체장의 승인없이 임의 처분 등이 해당된다. 시는 불법보조금 '
             "근절과 효율적인 점검 및 적극적인 시민관심을 유도하기 위해 '지방보조금 부정수급 신고센터(☏850-5031)'를 "
             '설치 운영하고 있다. 지방보조금 부정수급 신고 시 직접방문 및 국민신문고(www.epeople.or.kr), '
             '충주시홈페이지(www.chungju.or.kr)를 통해 접수하면 되고, 신고취지와 이유를 기재하고 부정행위와 관련한 '
             '증거자료를 제시하면 된다. 단, 익명 신고는 접수치 않

In [ ]:
print(raw_datasets)

Dataset({
    features: ['id', 'document', 'summary'],
    num_rows: 3012
})


In [ ]:
raw_datasets = raw_datasets.train_test_split(
    train_size=1-TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

## Data Pre-processing
모델을 학습하기전에 Hugging Face Transformers의 `Tokenizer` 라이브러리로 `MODEL_CHECKPOINT`에 저장되어 있는 토크나이저를 로딩하겠습니다.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

T5에서는 task를 prefix로 명시해주는 경우가 있습니다. `요약`이라는 prefix를 추가해줍니다.

In [ ]:
if MODEL_CHECKPOINT in ["psyche/KoT5-summarization", "t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "요약: "
else:
    prefix = ""

데이터 전처리를 위한 함수를 선언합니다.   
전처리 함수에서는 prefix 추가 및 tokenization을 진행합니다.  
이 과정에서 `token_tpye_ids`, `attention_mask도` 자동으로 생성됩니다.

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


이전에 생성했던 `dataset` 오브젝트에 `map` 메소드를 통해 전처리 함수를 적용합니다.  
`map`을 사용할 경우 `dataset`내에 쪼개져서 구성되어 있는 `train, validation, test` 데이터셋에도 한번에 적용됩니다. 

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Defining the model
이제 PLM 모델을 다운받고 finetuning을 해보겠습니다.   
요약 문제는 sequence-to-sequence task이기 때문에 `TFAutoModelForSeq2SeqLM` 클래스를 통해 모델을 로딩하겠습니다.   
`tokenizer`와 마찬가지로 `from_pretrained` 메소드를 통해 모델을 다운받을 수 있습니다.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, from_pt=True) # tf_model.h5 포멧의 경우는 from_pt=True 옵션 없어도 됨

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['lm_head.weight', 'encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Sequencce to Sequence 모델을 학습하기 위해서는 이에 맞는 data collator가 필요합니다.   
input뿐만 아니라 labels에 대해서도 padding처리를 해줘야 하기 때문입니다. 이를 위해 `DataCollatorForSeq2Seq` 를 사용할 수 있습니다.   
keras를 이용하고 있으므로 리턴 타입 `tf.Tensor`로 얻을 수 있도록 `return_tensors="tf"` 을 셋팅해줍니다.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

data_collator function을 적용해줍니다.  
추가적으로 `generation_dataset` 이라는 작은 사이즈의 데이터셋을 학습시에 계산할 `ROUGE` score를 위해 추가로 생성해줍니다.

In [ ]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

## Building and Compiling the model
이제 optimizer를 정의하고 모델을 컴파일 해줍니다. loss 계산은 내부적으로 처리됩니다.

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


## Training and Evaluating the model
학습을 진행하는 동안 동시에 모델을 평가하기 위해 `metric_fn`을 정의해서 ground-truth와 prediction 간의 `ROUGE` score를 계산해보겠습니다.

In [ ]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # masked label tokens 교체
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # 다른 metrics도 추가 가능하지만, 여기에서는 f1_score만 출력되도록 셋팅
    result = {"RougeL": result["f1_score"]}

    return result


이제 모델 학습을 시작해보겠습니다.

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# 오늘 실습에서는 test set을 validation set으로 사용해보겠습니다
model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

339/339 [==============================] - 590s 2s/step - loss: 1.2673 - val_loss: 1.1759 - RougeL: 0.3331


실습상 1 epoch만 돌리지만, 더 좋은 모델을 얻기 위해 최소 5 epoch정도 학습하는 것을 추천드립니다.

## Inference
이제 우리가 학습한 모델로 inference를 해보겠습니다. Huggingface의 `pipeline` 메소드에서 제공하는 `summarization`을 사용하면 쉽게 구현할 수 있습니다.   
`pipeline` 메소드에 학습한 모델과 토크나이저를 인자로 넣어줍니다. 모델이 TF로 학습되었으므로 `framework="tf"`도 함께 인자로 넣어줍니다.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summary = summarizer(
              raw_datasets["test"][0]["document"],
              min_length=MIN_TARGET_LENGTH,
              max_length=MAX_TARGET_LENGTH,
          )

pprint(f'document:{raw_datasets["test"][0]["document"]}')
pprint(f'label summary:{raw_datasets["test"][0]["summary"]}')
pprint(f'pred summary: {summary[0]["summary_text"]}')

("document:출처 : MBC에브리원 '비디오스타' '비디오스타'에 추억의 스타들이 모였다. 9일 방송된 MBC에브리원 '비디오스타'는 "
 "'개척자 특집, 방송가 콜럼버스의 재림'으로 꾸며져 김장훈, 브루노, 보쳉, 임은경이 출연했다. 이날 1999년 '남희석 이휘재의 한국이 "
 "보인다'로 뜨거운 인기를 얻은 1세대 외국인 방송인 브루노와 보쳉이 16년 만에 재회했다. 서로의 출연을 몰랐던 브루노와 보쳉은 "
 '스튜디오에서 만나자 서로를 껴안으며 반가움을 드러냈다. 브루노는 "보쳉은 너무 똑같다. 16년 만에 보는 것"이라고 기뻐했다. 브루노는 '
 '2002년에 비자 만료로 강제 출국했던 일화를 털어왔다. "안 좋은 일로 나가야 했는데 다 옛날이야기고 다시 잘 해보려고 한국에 '
 '왔다"라며 "이번에는 비자 확인 잘 했다"라고 전했다. 한국 활동 계획에는 "만약에 그렇게 된다면 연기와 예능을 하고 싶다. 오랜만에 '
 '한국에 왔는데 나를 기억하는 사람이 아직도 있으니까 너무 신기하고 앞으로 열심히 똑바로 하겠다"라고 각오를 전했다. 또 보쳉은 펀드 회사 '
 '대표와 인터넷 개발 회사 대표를 맡고 있다고 근황을 전했다. "옛날에는 펀드 회사에 있었다. 다른 사람의 돈을 받고 투자해줬다. 나중에는 '
 '프로젝트를 하고 싶어서 내가 하나 만들었다. 옛날에는 학원에서 영어, 중국어를 가르쳤지만 이제는 인터넷에서 가르친다"라고 설명했다. 이에 '
 '박나래와 김숙은 "정리를 하자면 투자회사가 있고 인터넷 사업체가 있는 것"이라며 정리했다. / 박유연')
("label summary:MBC에브리원 '비디오스타'는 9일 방송에서  '개척자 특집, 방송가 콜럼버스의 재림'이라는 제목으로 1999년 "
 "'남희석 이휘재의 한국이 보인다'로 인기를 끌었던 1세대 외국인 방송인 브루노와 보쳉의 16년만에 만남과 추억의 스타인 김장훈, 임은경이 "
 '출연했다.')
("pred summary: 9일 방송된 MBC에브리원 '비디오스타'에서 1999년 '남희석

아래와 같이 다양한 keyword arguments를 사용하면서 텍스트 생성을 위한 decoding 방법을 수정할 수 있습니다.

In [ ]:
summary = summarizer(
              raw_datasets["test"][0]["document"],
              min_length=MIN_TARGET_LENGTH,
              max_length=MAX_TARGET_LENGTH,
              do_sample=True,
              top_k=50,  
              top_p=0.92,
              temperature=2.5,
              num_return_sequences=4,
          )

print("pred summary:")
pprint([summ["summary_text"] for summ in summary])

pred summary:
["9일 방송된 MBC에브리원 '비디오스타'는 '개척자 특집, 방송가 콜럼버스의 재림'으로 김장훈, 브루노, 보쳉, 임은경이 출연했고 "
 "1999년 '남희석 이휘재의 한국이 보인다'로 뜨거운 인기를 얻은 1세대 외국인 방송인 브루노와 보쳉이 16년 만에 재회해 반가움을 "
 '드러냈다. ',
 "9일 방송된 MBC에브리원 '비디오스타'에서 1999년 '남희석 이휘재의 한국이 보인다'로 뜨거운 인기를 얻은 1세대 외국인 방송인 "
 '브루노와 보쳉이 16년 만에 재회했다. 보쳉은 펀드 회사 대표와 인터넷 개발 회사 대표를 맡고 있다고 근황을 전했다.',
 "9일 방송된 MBC에브리원 '비디오스타'는 '개척자 특집, 방송가 콜럼버스의 재림'으로 김장훈, 브루노, 보쳉, 임은경이 출연했고 1세대 "
 '외국인 방송인 브루노와 보쳉이 16년 만에 재회했다. ',
 "9일 방송된 MBC에브리원 '비디오스타'는 '개척자 특집, 방송가 콜럼버스의 재림'으로 꾸며져 김장훈, 브루노, 보쳉, 임은경이 "
 "출연했으며 1999년 '남희석 이휘재의 한국이 보인다'로 뜨거운 인기를 얻은 1세대 외국인 방송인 브루노와 보쳉이 16년 만에 재회해 "
 '서로를 껴안으며 반가움을 드러냈다.']


학습한 모델은 Hugging Face Model Hub에 모델명을 지정해서 업로드 할 수 있습니다.
  - `"your-username/the-name-you-picked"` 

```python
model.push_to_hub("transformers-qa", organization="keras-io")
tokenizer.push_to_hub("transformers-qa", organization="keras-io")
```
업로드한 모델은 아래와 같이 업로드 당시에 사용했던 모델명으로 다운받아서 사용할 수 있습니다.

```python
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("your-username/my-awesome-model")
```